In [ ]:
from scipy import stats, special, interpolate
import numpy as np
from pathlib import Path
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
import awkward as ak
import pandas as pd
import math
from sklearn import metrics
from damselfly.data import loaders

def background_cdf(x, N, var):
    return (1 - np.exp( - 1 * (x ** 2) / var)) ** (N)

def background_cdf_awk(x, n_signal, n_sample, var):
    
    N = n_sample - ak.to_numpy(n_signal)
    
    return (1 - np.exp( - 1 * (x ** 2) / var)) ** (N[:, np.newaxis])

def rician_pdf(x, var, loc_val):
    
    x = x[np.newaxis, :]
    loc_val = loc_val[:, np.newaxis]
    
    return x * np.exp( -1 * (x ** 2 + loc_val ** 2) / var ) * special.iv(0, x * 2 * loc_val / var)

def cdf_from_pdf(x, pdf):
    
    cdf = np.zeros((pdf.shape[0], x.size))
    for i in range(x.size):
        
        cdf[:, i] = np.trapz(pdf[:, 0:i+1], x[0:i+1], axis=-1)
        
    return cdf

def combine_batch_cdf(cdf, counts):
    
    count = 0
    cdf_ak = []
    for n in counts:
        cdf_ak.append(cdf[count:count+n, :])
        count+=n
    cdf_ak = ak.Array(cdf_ak)
        
    return ak.to_numpy(ak.prod(cdf_ak, axis=1))

def get_spectra_peaks(freq_spectra, threshold):
    
    peaks = []
    for i in range(freq_spectra.shape[0]):
        i_peak_inds = np.where(abs(freq_spectra[i, :]) > threshold)[0]
        i_peaks = abs(freq_spectra[i, i_peak_inds])

        peaks.append(list(i_peaks))
    
    return ak.Array(peaks)
    
def generate_signal_cdf(peaks, x, var, n_sample, batchsize=1000):
    
    peak_counts = ak.num(peaks, axis=1)
    num_spectra = ak.num(peaks, axis=0)
    
    spectra_inds = np.arange(0, num_spectra, 1)
    combined_cdf = np.zeros((num_spectra, x.size))
    
    for i, spectra_batch in enumerate(np.array_split(spectra_inds, num_spectra//batchsize)):
        
        batch_peaks = peaks[spectra_batch]
        batch_counts = ak.num(batch_peaks, axis=1)
        flat_batch_peaks = ak.to_numpy(ak.flatten(batch_peaks))
        
        batch_pdf = rician_pdf(x, var, flat_batch_peaks)
        batch_norm = 1/np.trapz(batch_pdf, x, axis=-1)
        batch_pdf = batch_norm[:, np.newaxis] * batch_pdf
        
        batch_cdf = cdf_from_pdf(x, batch_pdf)
        batch_cdf = combine_batch_cdf(batch_cdf, batch_counts)

        combined_cdf[spectra_batch, :] = batch_cdf * background_cdf_awk(x, batch_counts, n_sample, var)
        
        if i % 5 == 4:
            print(f'{i + 1} / {num_spectra//batchsize}')
            
    return combined_cdf

def estimate_min_pitch():
    
    path2signals = Path.home()/'group'/'project'/'results'/'beamforming'/'time_dependent'/'beamform_signals'
    file = '220505_dl_grid_84to90deg_1to45mm.h5'

    pitch_min=85.5
    pitch_max=90.0
    radius_min=0.005
    radius_max=0.005
    energy_val = 18550

    h5file = h5py.File(path2signals/file, 'r')
    pitch = h5file['pitch'][:]
    energy = h5file['energy'][:]
    radius = h5file['radius'][:]
    
    pitch_mask = np.logical_and(pitch<=pitch_max, pitch>=pitch_min)
    energy_mask = energy == energy_val
    radius_mask = radius == radius_min
    
    mask = pitch_mask * energy_mask * radius_mask
    

    select_pitch = np.sort(pitch[mask])
    signal_freq_spectra = []
    
    for ipitch in select_pitch:
        
        pitch_mask = pitch==ipitch
        energy_mask = energy == energy_val
        radius_mask = radius == radius_min
        mask = pitch_mask * energy_mask * radius_mask
        signal = h5file['x'][np.argwhere(mask).squeeze(), 0:8192]
        signal_freq_spectra.append(np.fft.fftshift(np.fft.fft(signal, axis=-1, norm='forward')))
    
    signal_freq_spectra = np.array(signal_freq_spectra)
    
    peak_threshold = 1e-8
    system_noise_temp = 10
    n_sample = 8192
    noise_variance = 1.38e-23 * 50 * 60 * 205e6 * system_noise_temp / n_sample
    n_pts_dist = 101
    x = np.linspace(0, 1e-6, n_pts_dist)

    peaks = get_spectra_peaks(signal_freq_spectra, peak_threshold)
    
    print(peaks)
    cdf = generate_signal_cdf(peaks, x, noise_variance, n_sample, batchsize=10)
    
    pdf = np.gradient(cdf, axis=-1)
    norm = 1/np.trapz(pdf, x)
    pdf = norm[:, np.newaxis] * pdf
    
    #pitch = pitch[mask]
    #inds = np.argsort(pitch)
    #cdf = cdf[inds, :]
    #pdf = pdf[inds, :]
    #signal_freq_spectra[inds, :]
    #pitch = np.sort(pitch)
    
    return cdf, pdf, select_pitch, signal_freq_spectra

def estimate_FFT(pitch):
    
    path2signals = Path.home()/'group'/'project'/'datasets'/'data'
    file = '220901_dl_test_data_85to90deg_5mm.h5'

    h5file = h5py.File(path2signals/file, 'r')

    metadata = {
        'energy':h5file['meta']['energy'][:],
        'pitch':h5file['meta']['theta_min'][:],
        'radius':h5file['meta']['x_min'][:]
    }

    metadata = pd.DataFrame(metadata)

    inds = np.array(metadata[
        (abs(metadata['pitch']-pitch)<0.0005)
    ].index.array)
    

    signal_freq_spectra = np.fft.fftshift(np.fft.fft(
        h5file['x'][inds, 0:8192], axis=-1, norm='forward'))
    
    nonzero_inds = np.argwhere(abs(signal_freq_spectra).sum(-1)>0).squeeze()
    signal_freq_spectra = signal_freq_spectra[nonzero_inds, :]
    signal_freq_spectra = signal_freq_spectra[0]
    
    peak_threshold = 1e-8
    system_noise_temp = 10
    n_sample = 8192
    noise_variance = 1.38e-23 * 50 * 60 * 205e6 * system_noise_temp / n_sample
    n_pts_dist = 101
    x = np.linspace(0, 1e-6, n_pts_dist)

    peaks = np.where(abs(signal_freq_spectra) > peak_threshold)[0]
    
    pdf = rician_pdf(x, noise_variance, abs(signal_freq_spectra[peaks]))
    norm_pdf = 1 / np.trapz(pdf, x, axis=-1)
    pdf = norm_pdf[:, np.newaxis] * pdf
    
    cdf = cdf_from_pdf(x, pdf)
    cdf = np.prod(cdf, axis=0)
    pdf = np.prod(pdf, axis=0)
    
    
    
    #cdf = generate_signal_cdf(peaks, x, noise_variance, n_sample)
    
    #pdf = np.gradient(cdf, axis=-1)
    #norm = 1/np.trapz(pdf, x)
    #pdf = norm[:, np.newaxis] * pdf
    
    return cdf, pdf

def estimate_mean_FFT(min_pitch = 86.0, max_pitch = 88.6):
    
    path2signals = Path.home()/'group'/'project'/'datasets'/'data'
    file = '220901_dl_test_data_85to90deg_5mm.h5'

    pitch_min=min_pitch
    pitch_max=max_pitch
    radius_min=0.005
    radius_max=0.005

    h5file = h5py.File(path2signals/file, 'r')

    metadata = {
        'energy':h5file['meta']['energy'][:],
        'pitch':h5file['meta']['theta_min'][:],
        'radius':h5file['meta']['x_min'][:]
    }

    metadata = pd.DataFrame(metadata)

    inds = np.array(metadata[
        (metadata['pitch']>=pitch_min)&(metadata['pitch']<=pitch_max)
        & (metadata['radius']>=radius_min)&(metadata['radius']<=radius_max)
    ].index.array)
    
    pitch_angles = np.array(metadata['pitch'].iloc[inds].array)

    signal_freq_spectra = np.fft.fftshift(np.fft.fft(
        h5file['x'][inds, 0:8192], axis=-1, norm='forward'))
    
    nonzero_inds = np.argwhere(abs(signal_freq_spectra).sum(-1)>0).squeeze()
    signal_freq_spectra = signal_freq_spectra[nonzero_inds, :]
    pitch_angles = pitch_angles[nonzero_inds]
    
    peak_threshold = 1e-8
    system_noise_temp = 10
    n_sample = 8192
    noise_variance = 1.38e-23 * 50 * 60 * 205e6 * system_noise_temp / n_sample
    n_pts_dist = 101
    x = np.linspace(0, 1e-6, n_pts_dist)

    peaks = get_spectra_peaks(signal_freq_spectra, peak_threshold)
    cdf = generate_signal_cdf(peaks, x, noise_variance, n_sample)
    
    pdf = np.gradient(cdf, axis=-1)
    norm = 1/np.trapz(pdf, x)
    pdf = norm[:, np.newaxis] * pdf
    
    return cdf, pdf

def estimate_MF(N_template, match=1,):
    
    path2scores = Path.home()/'group'/'project'/'results'/'matched_filter'/\
    'scores'/'220922_dl_test_data_mf_scores.npz'
    
    pitch_min=86.0
    pitch_max=88.
    
    scores_file = np.load(path2scores)
    
    #print([*scores_file.keys()])
    
    ideal_scores_signal = scores_file['signal_ideal_scores']
    
    signal_metadata = {
        'signal_energy':scores_file['signal_energy'][:],
        'signal_pitch':scores_file['signal_pitch'][:],
    }
    
    template_metadata = {
        'template_energy':scores_file['template_energy'][:],
        'template_pitch':scores_file['template_pitch'][:],
    }

    signal_metadata = pd.DataFrame(signal_metadata)
    template_metadata = pd.DataFrame(template_metadata)

    signal_inds = np.array(signal_metadata[
        (signal_metadata['signal_pitch']>=pitch_min)&(signal_metadata['signal_pitch']<=pitch_max)
    ].index.array)
    
    template_inds = np.array(template_metadata[
        (template_metadata['template_pitch']>=pitch_min)&(template_metadata['template_pitch']<=pitch_max)
    ].index.array)
    
    #print(ideal_scores_signal.shape)
    ideal_scores_signal = ideal_scores_signal[signal_inds]
    ideal_scores_signal = ideal_scores_signal[~np.isnan(ideal_scores_signal)]
    
    x = np.linspace(0, 12, 201)

    ideal_signal_cdf = stats.rice.cdf(
        x[np.newaxis, :],
        match * ideal_scores_signal[:, np.newaxis],
        loc = match * ideal_scores_signal[:, np.newaxis] * (1 - 1 / math.sqrt(2)),
        scale=1/math.sqrt(2)
    )
    
    noise_cdf = stats.rayleigh.cdf(x, loc=0, scale=(1/math.sqrt(2))) ** ((N_template-1)/2)
    noise_cdf = noise_cdf[np.newaxis, :]
    
    signal_cdf = noise_cdf * ideal_signal_cdf
    
    return signal_cdf

def best_MF(match=1):
    
    path2scores = Path.home()/'group'/'project'/'results'/'matched_filter'/\
    'scores'/'220822_dl_test_data_mf_scores.npz'
    
    scores_file = np.load(path2scores)
    
    #print([*scores_file.keys()])

    best_scores = scores_file['scores'].max(axis=0)
    best_sf = stats.rice.sf(
        x[np.newaxis, :],
        best_scores[:, np.newaxis],
        loc = best_scores[:, np.newaxis] * (1 - 1 / math.sqrt(2)),
        scale=1/math.sqrt(2)
    )
    
    return best_sf
    
    
def estimate_NN():
    path2outputs = Path.home()/'group'/'project'/'scripting'/'output'/\
    '220929_eval_cnn1d'/'model1'/'model1.npz'
    path2ind = Path.home()/'group'/'project'/'scripting'/'output'/\
    '220929_eval_cnn1d'/'model1'/'train_inds.npy'
    path2originaldata = Path.home()/'group'/'project'/'datasets'/'data'/'220901_dl_test_data_85to90deg_5mm.h5'
    originaldata = h5py.File(path2originaldata, 'r')
    pitch_angles = originaldata['meta']['theta_min'][np.load(path2ind)]
    n_pitch = pitch_angles.size
    originaldata.close()
    pitch_angles = np.tile(pitch_angles, 3)
    pitch_angles = np.concatenate((pitch_angles, np.zeros(pitch_angles.size)), axis=0)
    output_file = np.load(path2outputs)
    

    combined_output = []
    combined_pitch = []
    ep_output_shape = ()
    for i, key in enumerate(output_file):
        combined_output.extend(output_file[key])
        combined_pitch.extend(pitch_angles)
        ep_output_shape = output_file[key].shape
        #print(ep_output_shape)
        
    combined_output = np.array(combined_output)
    combined_pitch = np.array(combined_pitch)
    combined_output = special.softmax(combined_output, axis=-1)
    
    labels = np.zeros(ep_output_shape[0], dtype=np.int32)
    labels[0:ep_output_shape[0]//2]=1
    labels = np.tile(labels, 100)
    
    fpr, tpr, _ = metrics.roc_curve(labels, combined_output[:, 1])
    
    return labels, combined_output, combined_pitch

def estimate_NN2():
    path2outputs = Path.home()/'group'/'project'/'scripting'/'output'/\
    '220902_eval_cnn1d'/'model2'/'model2.npz'
    output_file = np.load(path2outputs)
    
    
    combined_output = []
    ep_output_shape = ()
    for i, key in enumerate(output_file):
        combined_output.extend(output_file[key])
        ep_output_shape = output_file[key].shape
        
    combined_output = np.array(combined_output)
    combined_output = special.softmax(combined_output, axis=-1)
    
    labels = np.zeros(ep_output_shape[0], dtype=np.int32)
    labels[0:ep_output_shape[0]//2]=1
    labels = np.tile(labels, 100)
    
    fpr, tpr, _ = metrics.roc_curve(labels, combined_output[:, 1])
    
    return fpr, tpr
    

def match_est_MF():
    
    path2scores = Path.home()/'group'/'project'/'results'/'matched_filter'/\
    'scores'/'220922_dl_test_data_mf_scores.npz'
    
    pitch_min=86.0
    pitch_max=88.6
    
    scores_file = np.load(path2scores)
    
    print([*scores_file.keys()])
    
    ideal_scores_signal = scores_file['signal_ideal_scores']
    #scores = scores_file['scores']
    
    signal_metadata = {
        'signal_energy':scores_file['signal_energy'][:],
        'signal_pitch':scores_file['signal_pitch'][:],
    }
    
    template_metadata = {
        'template_energy':scores_file['template_energy'][:],
        'template_pitch':scores_file['template_pitch'][:],
    }

    signal_metadata = pd.DataFrame(signal_metadata)
    template_metadata = pd.DataFrame(template_metadata)

    signal_inds = np.array(signal_metadata[
        (signal_metadata['signal_pitch']>=pitch_min)&(signal_metadata['signal_pitch']<=pitch_max)
    ].index.array)
    
    template_inds = np.array(template_metadata[
        (template_metadata['template_pitch']>=pitch_min)&(template_metadata['template_pitch']<=pitch_max)
    ].index.array)
    
    print(scores_file['scores'].shape, ideal_scores_signal.shape)
    ideal_scores_signal = ideal_scores_signal[signal_inds]
    #template_inds, signal_inds = np.meshgrid(template_inds, signal_inds)
    #scores = scores[signal_inds, template_inds]
    
    no_nan_signal = ~np.isnan(ideal_scores_signal)
    ideal_scores = ideal_scores_signal[no_nan_signal]
    
    shape = scores_file['scores'].shape
    
    #num_indicies = np.arange(100, shape[1]//50, 50) # the number of templates
    num_indicies = np.int32(np.logspace(2, 5, 21)) # the number of templates
    print(num_indicies)
    mean_match = np.zeros(num_indicies.size) 
    for i, number in enumerate(num_indicies):
        indicies = np.int32(np.linspace(0, shape[1]-1, number))
        signal_grid, template_grid = np.meshgrid(signal_inds[no_nan_signal], indicies)
        
        select_scores = scores_file['scores'][signal_grid, template_grid] # max over the templates
        #print(select_scores.shape)
        #print(np.where(np.isnan(select_scores))[0].shape)
        
        temp_where = np.where(np.isnan(select_scores))
        select_scores[temp_where[0], temp_where[1]] = 0
        
        select_scores = select_scores.max(axis=0)
        
        match = np.mean(select_scores / ideal_scores)
        mean_match[i] = match
        
        #if i % 5 == 4:
        print(f'{i+1} / {num_indicies.size}. Match = {match}, Templates = {number}')
            
    
    return mean_match, num_indicies
        
        
    


# minimum pitch angle

In [ ]:

cdf, pdf, pitch, spectra = estimate_min_pitch()

#print(cdf.shape, pitch.shape)

#print(np.argsort(pitch))

#print(np.where(pitch==89.975))


#cdf = cdf[np.flip(np.argsort(pitch)), :]
#pdf = pdf[np.flip(np.argsort(pitch)), :]
#spectra = spectra[np.flip(np.argsort(pitch)), :]
#pitch = np.flip(np.sort(pitch))


print(pitch.size)

system_noise_temp = 10
n_pts_dist = 101
n_sample = 8192
noise_variance = 1.38e-23 * 50 * 60 * 205e6 * system_noise_temp / n_sample
x = np.linspace(0, 1e-6, n_pts_dist)
cdf_background = background_cdf(x, n_sample, noise_variance)
pdf_background = np.gradient(cdf_background) / np.trapz(np.gradient(cdf_background), x)


sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
for i in range(50,65,2):
    
    ax.plot(1-cdf_background, 1-cdf[i, :], label=f'{pitch[i]}')
    
ax.legend(loc=2, title='Pitch Angle')
ax.set_xscale('log')
ax.set_xlim(1e-4, 1)
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curves for FFT Power Threshold')

plt.savefig(Path.home()/'group'/'project'/'plots'/'220831_fft_power_threshold_roc_transistion')

#sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)
#fig = plt.figure(figsize=(13, 8))
#ax = fig.add_subplot(1,1,1)
#for i in range(0,9,1):
    
#    ax.plot(1-cdf_background, 1-cdf[i, :], label=f'{pitch[i]}')
    
#ax.legend(loc=4)
#ax.set_xscale('log')
#ax.set_xlim(1e-4, 1)

#sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)
#fig = plt.figure(figsize=(13, 8))
#ax = fig.add_subplot(1,1,1)
#for i in [0, 90]:
    
#    ax.plot(abs(spectra[i]), label=f'{pitch[i]}')
    
#ax.legend(loc=4)
#ax.set_xscale('log')
#ax.set_xlim(1e-4, 1)

#print(np.argwhere(abs((1-cdf_background - 1e-2))<3e-3))


# Estimate FFT

In [ ]:

system_noise_temp = 10
n_pts_dist = 101
n_sample = 8192
noise_variance = 1.38e-23 * 50 * 60 * 205e6 * system_noise_temp / n_sample
x = np.linspace(0, 1e-6, n_pts_dist)
min_pitch = 86
max_pitch = 88.

cdf_background = background_cdf(x, n_sample, noise_variance)
pdf_background = np.gradient(cdf_background) / np.trapz(np.gradient(cdf_background), x)

cdf_mean, pdf_mean = estimate_mean_FFT(min_pitch=min_pitch, max_pitch=max_pitch)

In [ ]:
sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)
fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

system_noise_temp = 10
n_pts_dist = 101
n_sample = 8192
noise_variance = 1.38e-23 * 50 * 60 * 205e6 * system_noise_temp / n_sample
x = np.linspace(0, 1e-6, n_pts_dist)

pitch_list = [89, 88.5, 88, 87, 86]
pitch_labels = [89.0, 88.5, 88.0, 87.0, 86.0]

for i, pitch in enumerate(pitch_list):
    cdf, pdf, = estimate_FFT(pitch)
    auc = round(abs(np.trapz(1-cdf*cdf_background, 1-cdf_background)),2)
    ax.plot(1-cdf_background, 1-cdf*cdf_background, label=rf'Pitch Angle={pitch_labels[i]}$^\circ$, AUC={auc}')
    
auc = round(abs(np.trapz(1-cdf_mean.mean(axis=0), 1-cdf_background)),2)
ax.plot(1-cdf_background, 1-cdf_mean.mean(axis=0), color='k', label=f'Average ROC, AUC={auc}')
#ax.set_xscale('log')
#ax.set_xlim(1e-6, 1)
ax.legend(loc=4)
ax.set_ylabel('True Positive Rate')
ax.set_xlabel('False Positive Rate')
#plt.savefig(Path.home()/'group'/'project'/'plots'/'220929_fft_power_threshold_roc_by_pitch', dpi=128)

In [ ]:
#sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)
#fig = plt.figure(figsize=(13,8))
#ax = fig.add_subplot(1,1,1)

#ax.plot(x, cdf)
#ax.plot(x, cdf * cdf_background)
#ax.plot(x, background_cdf(x, 100, 1))



In [ ]:
abs(pitch_angles-target_pitch)

In [ ]:
sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)
fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

pitch_list = [89.8]
inds = []
for target_pitch in pitch_list:
    where_pitch = np.where(abs(pitch_angles-target_pitch)<0.0005)
    print(where_pitch)
    inds.append(where_pitch[0][0])

for ind in inds:
    ax.plot(1-cdf_background, 1-cdf[ind, :], label='FFT')
ax.plot(np.linspace(0, 1, 1001), np.linspace(0,1,1001), ':', color='k')
ax.legend(loc=4)
#ax.set_xscale('log')
#ax.set_xlim(1e-4, 1)


# Estimate MF

In [ ]:
x = np.linspace(0, 12, 201)
N = 10000
noise_sf = 1 - stats.rayleigh.cdf(x, loc=0, scale=(1/math.sqrt(2))) ** (1)
noise_sf_sqrt_trial = 1 - stats.rayleigh.cdf(x, loc=0, scale=(1/math.sqrt(2))) ** (np.sqrt(N))
noise_sf_halfn_trial = 1 - stats.rayleigh.cdf(x, loc=0, scale=(1/math.sqrt(2))) ** (N/2)
noise_sf_logn_trial = 1 - stats.rayleigh.cdf(x, loc=0, scale=(1/math.sqrt(2))) ** (np.log(N))

#signal_sf = stats.rice.sf(x, score, loc=score*(1 - 1 / math.sqrt(2)), scale=1/math.sqrt(2))

mf_cdf_list = []
match_list = np.flip([0.5, 0.75, 0.85, 0.9, 0.95])
N_template_list = np.flip([2250, 12500, 27500, 40000, 90000])
for i in range(len(match_list)):
    print(match_list[i])
    mf_cdf_list.append(estimate_MF(N_template_list[i], match=match_list[i]))

In [ ]:
noise_pdf = stats.rayleigh.pdf(x, loc=0, scale=(1/math.sqrt(2)))
noise_cdf = stats.rayleigh.cdf(x, loc=0, scale=(1/math.sqrt(2)))
sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)
fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

ax.plot(x, noise_cdf ** (1))
ax.plot(x, background_cdf(x, 100, 1))
#ax.plot(x, 1 - noise_cdf ** (1000000))


In [ ]:
sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)
fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)


for i in range(len(match_list)):
    noise_sf = 1 - stats.rayleigh.cdf(x, loc=0, scale=(1/math.sqrt(2))) ** (N_template_list[i]/2)
    
    auc = abs(np.trapz(1-mf_cdf_list[i].mean(axis=0),1 - stats.rayleigh.cdf(x, loc=0, scale=(1/math.sqrt(2))) ** (N_template_list[i]/2)))
    
    ax.plot(noise_sf, 1-mf_cdf_list[i].mean(axis=0), label=f'{match_list[i]}, AUC={round(auc,2)}')
    
    
#ax.plot(noise_sf_logn_trial, mf_sf_list[0].mean(axis=0), label='MF with log(n) Trial')
#ax.plot(noise_sf_sqrt_trial, mf_sf_list[0].mean(axis=0), label='MF with Sqrt(n) Trial')
#ax.plot(noise_sf_halfn_trial, mf_sf_list[0].mean(axis=0), label='MF with n/2 Trial')

#ax.plot(1-cdf_background, 1-cdf.mean(axis=0), label='FFT')
auc = abs(np.trapz(1-cdf_mean.mean(axis=0), 1-cdf_background))
ax.plot(1-cdf_background, 1-cdf_mean.mean(axis=0), color='k', label=f'FFT, AUC={round(auc, 2)}')
ax.plot(np.linspace(0, 1, 1001), np.linspace(0,1,1001), ':', color='k')
ax.legend(loc=4, title='Mean Match')

ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')

#plt.savefig(Path.home()/'group'/'project'/'plots'/'220929_mf_roc_by_match', dpi=128)
#ax.set_xscale('log')
#ax.set_xlim(1e-4, 1)


In [ ]:
best_sf = best_MF()

In [ ]:
mean_match, num_indicies = match_est_MF()

In [ ]:
sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)
cmap = sns.color_palette('turbo_r', 5)
clist = sns.color_palette('deep', 10)
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(num_indicies, mean_match, '.')
ax.set_xlabel('Template Number')
ax.set_ylabel('Mean Match')
ax.set_title('Expected Match vs Template Number')
ax.set_xscale('log')

plt.savefig(Path.home()/'group'/'project'/'plots'/'220927_match_vs_template_number.png', dpi=128)

In [ ]:
print(mean_match)

# Estimate NN

In [ ]:
labels, output, pitch = estimate_NN()
#nn2_fpr, nn2_tpr = estimate_NN2()

In [ ]:
nn_fpr_by_pitch = []
nn_tpr_by_pitch = []

for temp_pitch in [86., ]:#88.2, 88.1, 88.0, 87.9, 87.8, 87.7, 87.6, 87.5]:

    inds = ((pitch>=temp_pitch)*(pitch<=temp_pitch+2.0)+(pitch==0.0))
    nn_fpr_temp, nn_tpr_temp, _ = metrics.roc_curve(labels[inds], output[inds, 1])
    
    nn_fpr_by_pitch.append(nn_fpr_temp)
    nn_tpr_by_pitch.append(nn_tpr_temp)

nn_fpr_all, nn_tpr_all, _ = metrics.roc_curve(labels, output[:, 1])



In [ ]:
sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)
cmap = sns.color_palette('turbo', n_colors=5)
clist = sns.color_palette('deep', n_colors=10)
fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)


#ax.plot(nn_fpr_all, nn_tpr_all, label='NN')

    
for i in [0, 1, 2]:
    noise_sf_mf = 1 - stats.rayleigh.cdf(x, loc=0, scale=(1/math.sqrt(2))) ** (N_template_list[i]/2)
    sf_mf = 1-mf_cdf_list[i].mean(axis=0)
    ax.plot(noise_sf_mf, sf_mf, label=f'MF, Match={100*np.flip(match_list[i])}%', color=cmap[i])
#ax.plot(1-cdf_background, 1-cdf.mean(axis=0), label='FFT')
ax.plot(np.linspace(0, 1, 1001), np.linspace(0,1,1001), ':', color='k')
ax.plot(1-cdf_background, 1-cdf_mean.mean(axis=0), color='k', label='FFT')

for i in range(len(nn_fpr_by_pitch)):
    ax.plot(nn_fpr_by_pitch[i], nn_tpr_by_pitch[i], label='NN', color=clist[3], lw=5)
    
ax.legend(loc=4)
#ax.set_xscale('log')
#ax.set_xlim(1e-4, 1)

ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')

#plt.savefig(Path.home()/'group'/'project'/'plots'/'220930_nn_roc_vs_mf', dpi=128)

# Compare MF to NN to FFT

In [ ]:
sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)
cmap = sns.color_palette('turbo_r', 5)
clist = sns.color_palette('deep', 10)
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)


for i, sf in enumerate(mf_sf_list): 
    ax.plot(noise_sf, mf_sf_list[i].mean(axis=0), color=cmap[i], lw=3,
            label=f'Match={np.round(match_list[i],1)}')
    
#ax.plot(noise_sf, best_sf.mean(axis=0), color='k')
ax.plot(nn_fpr, nn_tpr, color='k', lw=4, label='NN')
ax.plot(1-cdf_background, 1-cdf.mean(axis=0), ':', color='k', lw=4, label='FFT')
ax.plot(np.linspace(0, 1, 10001), np.linspace(0, 1, 10001), color='grey', label='Random')

ax.legend(loc=2)
ax.set_xscale('log')
ax.set_xlim(1e-4, 1)

ax.set_title('ROC Curve Comparison')
ax.set_ylabel('True Positive Rate')
ax.set_xlabel('False Positive Rate')

#plt.savefig(Path.home()/'group'/'project'/'plots'/'220822_roc_curve_comparison.png', dpi=128)

In [ ]:
1-cdf.mean(axis=0)[0:86]

In [ ]:
f_interp = interpolate.interp1d(nn_fpr, nn_tpr)

sns.set_theme(context='talk', style='whitegrid', font_scale=1.2)
cmap = sns.color_palette('inferno', 5)
clist = sns.color_palette('deep', 10)
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(1-cdf_background[0:86], f_interp(1-cdf_background[0:86]) / (1-cdf.mean(axis=0)[0:86]), lw=4)
ax.set_xscale('log')
ax.set_xlim(1e-4, 1)
#ax.set_yscale('log')
ax.set_ylim(0, 12)

ax.set_title('True Positive Rate Ratio')
ax.set_xlabel('FPR')
ax.set_ylabel(r'$\mathrm{TPR}_{\mathrm{NN}}\:/\;\mathrm{TPR}_{\mathrm{FFT}}$')

#plt.savefig(Path.home()/'group'/'project'/'plots'/'220822_tpr_ratio.png', dpi=128)


In [ ]:
plt.plot(mean_match)

# Validation

In [ ]:
test_index = 1000
n_mc = 10000
noise = np.random.multivariate_normal([0, 0], np.eye(2) * noise_variance / 2, n_sample)
noise = noise[:, 0] + 1j * noise[:, 1]

sns.set_theme(context='talk', style='whitegrid', palette='bright')

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
ax.plot(x, cdf[test_index, :])
ylims = ax.get_ylim()

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
ax.plot(x, np.gradient(cdf[test_index, :]) / np.trapz(np.gradient(cdf[test_index, :]), x))
ylims = ax.get_ylim()

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
ax.plot(abs(noise))
ax.plot(abs(signal_freq_spectra[test_index, :]))

test_peak_inds = np.where(abs(signal_freq_spectra[test_index, :])>peak_threshold)[0]
test_max_val = np.zeros(n_mc)

noise_only_max_val = np.zeros(n_mc)
for i in range(n_mc):
    
    noise = np.random.multivariate_normal([0, 0], np.eye(2) * noise_variance / 2, n_sample)
    noise = noise[:, 0] + 1j * noise[:, 1]
    
    test_max_val[i] = np.max(abs(noise + signal_freq_spectra[test_index, :])[:])
    
for i in range(n_mc):
    
    noise = np.random.multivariate_normal([0, 0], np.eye(2) * noise_variance / 2, n_sample)
    noise = noise[:, 0] + 1j * noise[:, 1]
    
    noise_only_max_val[i] = np.max(abs(noise))
    
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
hist = ax.hist(test_max_val, 201, density=True, histtype='step')
hist = ax.hist(noise_only_max_val, 201, density=True, histtype='step')
ax.plot(x, np.gradient(cdf[test_index, :]) / np.trapz(np.gradient(cdf[test_index, :]), x))
ax.set_yscale('log')
#ax.set_ylim(1e3, 1e7)
  
    

In [ ]:
y_cdf.shape

In [ ]:
test_peaks_per_signal = ak.num(test_signal_peaks)

cdf_ak = []
count = 0
for n_peaks in test_peaks_per_signal:
    cdf_ak.append(y_cdf[count:count+n_peaks, :])
    count += n_peaks
cdf_ak = ak.Array(cdf_ak)



In [ ]:
ak.prod(cdf_ak, axis=1)

In [ ]:
np.prod(y_cdf[0:peaks_per_signal[0], :], axis=0).shape

In [ ]:
np.array_split(np.linspace(0, 100), 10)

In [ ]:
x = ak.Array([1,1,1,11,1,1,1,1,1,1,1,1])

In [ ]:
np.array_split(x, 3)